# Trial Runs

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings




Total pages loaded: 41


In [ ]:
page = pages[4]
page.metadata

{'producer': 'Adobe PDF Library 17.0',
 'creator': 'Adobe InDesign 18.0 (Macintosh)',
 'creationdate': '2022-12-13T16:08:00-05:00',
 'moddate': '2023-01-04T20:50:56+02:00',
 'trapped': '/False',
 'source': 'C:/Users/KUSHAL/Downloads/pdf_for_rag.pdf',
 'total_pages': 41,
 'page': 4,
 'page_label': '5'}

## Task 1

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY_temp_1")


loader = PyPDFLoader("C:/Users/KUSHAL/Downloads/pdf_for_rag.pdf")
pages = loader.load()
print(f"Total pages loaded: {len(pages)}")

# Step 1: Create text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)

# Step 2: Split pages into chunks WITH metadata
documents = text_splitter.split_documents(pages)

# Step 3: Create embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Step 4: Store chunks into Chroma with metadata
vectordb = Chroma.from_documents(documents, embedding_model, persist_directory="./chroma_db")

# Optional: Persist to disk
vectordb.persist()

from langchain.chains import RetrievalQA
from langchain.llms import OpenAI  # or ChatOpenAI, ChatGroq, etc.

retriever = vectordb.as_retriever()

llm = ChatGroq(model_name="qwen/qwen3-32b", api_key=GROQ_API_KEY)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=retriever,
    return_source_documents=True
)

query = "What are the steps to career growth?"
result = qa_chain({"query": query})

# Print the answer
print("Answer:", result["result"])

# Print cited page numbers
for doc in result["source_documents"]:
    print(f"Cited from page: {doc.metadata.get('page_label')}")



Total pages loaded: 41
Answer: <think>
Okay, the user is asking about the steps to career growth. Let me check the provided context.

Looking at the pages, PAGE 5 and 6 mention "Three Steps to Career Growth" in Chapter 1. The text says the three key steps are learning foundational skills, working on projects (to... [the rest is cut off]). The next part on PAGE 6 talks about the rise of AI and careers in that field, but the main points are the three steps. The user's question is general, not specific to AI, but the context given is about AI. However, the answer should be based on the provided info. The three steps listed are:

1. Learning foundational skills
2. Working on projects
3. The third step is mentioned in the title but not fully in the provided text. Wait, the first page says "Three Steps to Career Growth CHAPTER 1" and then later it lists "Three key steps... are learning foundational skills, working on projects (to..." but the third part is cut off. Hmm. The user might need th

## Task 2 - Web Scraping

In [22]:
from langchain_community.document_loaders import WebBaseLoader

# Step 1: Load a website
loader = WebBaseLoader("https://techifysolutions.com/blogs/")

# Step 2: Load and print the document
documents = loader.load()

print("Number of documents loaded:", len(documents))
print("Sample content:\n", documents[0].page_content[:1000])


Number of documents loaded: 1
Sample content:
 Blog - Techify - IT Solutions Partner                                                 Schedule Meeting        What We DoSoftware Product DevelopmentWeb App DevelopmentMobile App DevelopmentCloud  ServicesDevOps and AutomationSoftware Architecture and Re-architecture ServicesAPI DevelopmentUI/UX DesignApplication Testing ServicesDigital TransformationCRM ServicesAI/ML ServicesIT Advisory and ConsultingStaff AugmentationApplication MaintenanceLet's grow together Partner with us Explore All Follow UsLinkedin Email Usinfo@techifysolutions.com Marketing / Sales+91 72030 54040 HR / Job / Vacancy+91 78620 63131SolutionsCloudLaunchPadBizioCloudLaunchPadBizioWalkinsAI CardVaultWalkinsAI CardVaultLet's grow together Partner with us Explore All Follow UsLinkedin Email Usinfo@techifysolutions.com Marketing / Sales+91 72030 54040 HR / Job / Vacancy+91 78620 63131TechnologyFrontendAngularReact JSMobileReact NativeFlutterAndroidIOSBackendJava with Spring

In [23]:
pip install langchain langchain_community langchain_hyperbrowser html2text chromadb sentence-transformers



   -------------------- ------------------- 2/4 [hyperbrowser]
   ------------------------------ --------- 3/4 [langchain_hyperbrowser]
   ---------------------------------------- 4/4 [langchain_hyperbrowser]

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from langchain_hyperbrowser import HyperbrowserCrawlTool, HyperbrowserScrapeTool
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
import os
from dotenv import load_dotenv
import traceback

# 🔹 Load environment variables from .env
load_dotenv()
api_key = os.getenv("HYPERBROWSER_API_KEY")

if not api_key:
    raise EnvironmentError("❌ HYPERBROWSER_API_KEY is missing from environment. Please check your .env file.")

# 🔹 Set API key for Hyperbrowser
os.environ["HYPERBROWSER_API_KEY"] = api_key

# 🔹 1. Crawl the Techify website
print("🌐 Crawling Techify website...")
crawl_tool = HyperbrowserCrawlTool()
crawl_result = crawl_tool.invoke({
    "url": "https://techifysolutions.com/",
    "max_pages": 1
})

# 🔍 Extract URLs from 'data' key inside crawl_result
if isinstance(crawl_result, dict) and "data" in crawl_result:
    data = crawl_result["data"]
    urls = [page.metadata["url"] for page in data if "url" in page.metadata]
else:
    print("❌ Unexpected crawl result format:", crawl_result)
    urls = []

print(f"🔗 Found {len(urls)} URLs to scrape.")

# 🔹 2. Scrape each page to get clean text
print("🧹 Scraping individual pages...")
scrape_tool = HyperbrowserScrapeTool()
documents = []

for url in urls:
    try:
        result = scrape_tool.invoke({
            "url": url,
            "scrape_options": {
                "formats": ["markdown", "html"],
                "include_tags": ["h1", "h2", "p", "li"]
            },
            "session_options": {
                "solve_captchas": True
            }
        })

        job_data = result.get("data", None)
        # ✅ Prefer HTML, fallback to Markdown
        content = (getattr(job_data, "html", None) or getattr(job_data, "markdown", None) or "").strip()

        if not content:
            print(f"⚠️ No content returned for: {url}")
            print(f"🔍 Raw result: {result}")
        else:
            print(f"✅ Scraped: {url}")
            documents.append(Document(page_content=content, metadata={"source": url}))

    except Exception as e:
        print(f"❌ Exception while scraping {url}: {e}")
        traceback.print_exc()

print(f"📄 Total documents scraped: {len(documents)}")

# ✅ Prevent crash if no content was scraped
if not documents:
    print("❌ No documents to process. Exiting.")
    exit()

# 🔹 3. Split the documents into chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)
print(f"🧩 Total chunks created: {len(chunks)}")

# ✅ Prevent crash if no chunks
if not chunks:
    print("❌ No chunks to embed. Exiting.")
    exit()

# 🔹 4. Embed and store in Chroma vector DB
print("🧠 Embedding and storing in Chroma DB...")
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

vectordb = Chroma.from_documents(chunks, embedding_model, persist_directory="./techify_chroma_db")
vectordb.persist()

print("✅ Vector store created and saved at './techify_chroma_db'")


c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\pydantic\_internal\_fields.py:198: UserWarning: Field name "schema" in "ExtractArgs" shadows an attribute in parent "BaseModel"
  warnings.warn(


🌐 Crawling Techify website...
🔗 Found 1 URLs to scrape.
🧹 Scraping individual pages...
❌ Exception while scraping https://techifysolutions.com/: Free plan does not support use of proxies or solving captchas. Please upgrade your plan to enable these features. - Status: 402 - Caused by HTTPStatusError: Client error '402 Payment Required' for url 'https://app.hyperbrowser.ai/api/scrape'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/402
📄 Total documents scraped: 0
❌ No documents to process. Exiting.
🧩 Total chunks created: 0
❌ No chunks to embed. Exiting.
🧠 Embedding and storing in Chroma DB...


Traceback (most recent call last):
  File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\hyperbrowser\transport\sync.py", line 16, in _handle_response
    response.raise_for_status()
  File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\httpx\_models.py", line 829, in raise_for_status
    raise HTTPStatusError(message, request=request, response=self)
httpx.HTTPStatusError: Client error '402 Payment Required' for url 'https://app.hyperbrowser.ai/api/scrape'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/402

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\KUSHAL\AppData\Local\Temp\ipykernel_23964\510364009.py", line 45, in <module>
    result = scrape_tool.invoke({
             ^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\langchain_core\tools\base.py", line 599, in invoke
    return self.run(tool_input, **kwargs)
  

ValueError: Expected Embeddings to be non-empty list or numpy array, got [] in upsert.

: 

In [18]:
pip install faiss-cpu

   ---------------------------------------- 0.0/14.9 MB ? eta -:--:--
   ------- -------------------------------- 2.6/14.9 MB 18.9 MB/s eta 0:00:01
   ---------------- ----------------------- 6.3/14.9 MB 18.4 MB/s eta 0:00:01
   ------------------------------- -------- 11.8/14.9 MB 20.5 MB/s eta 0:00:01
   ---------------------------------------  14.7/14.9 MB 21.5 MB/s eta 0:00:01
   ---------------------------------------- 14.9/14.9 MB 15.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import time
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
from urllib.parse import urlparse
from datetime import datetime
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.schema import Document
from langchain_hyperbrowser import HyperbrowserCrawlTool

# 📦 Load environment variables
load_dotenv()
api_key = os.getenv("HYPERBROWSER_API_KEY")
if not api_key:
    raise EnvironmentError("❌ Missing HYPERBROWSER_API_KEY in .env")
os.environ["HYPERBROWSER_API_KEY"] = api_key

# 🔧 Toggle crawling
use_crawling = False

# 🔗 Step 1: Define URLs
if use_crawling:
    print("🌐 Crawling Techify website...")
    crawl_tool = HyperbrowserCrawlTool()
    crawl_result = crawl_tool.invoke({
        "url": "https://techifysolutions.com/industries/",
        "max_pages": 3
    })

    urls = []
    if isinstance(crawl_result, dict) and "data" in crawl_result:
        for page in crawl_result["data"]:
            url = page.metadata.get("url")
            if url:
                urls.append(url)
    else:
        print("❌ Unexpected crawl result:", crawl_result)

    if not urls:
        print("❌ No URLs found. Exiting.")
        exit()
else:
    # 📝 Manual URLs
    urls = [
        "https://techifysolutions.com/industries/",
        "https://techifysolutions.com/industries/information-technology/",
        "https://techifysolutions.com/industries/real-estate/",
        "https://techifysolutions.com/industries/retail/",
        "https://techifysolutions.com/industries/automobiles/",
        "https://techifysolutions.com/industries/healthcare-and-wellness/",
    ]

print(f"🔗 Using {len(urls)} URLs:")
for u in urls:
    print("  •", u)

# 🧹 Step 2: Scrape with enriched metadata
def scrape_content(urls, delay=1, snippet_len=500):
    session = requests.Session()
    session.headers.update({"User-Agent": "Mozilla/5.0"})
    documents = []

    for idx, url in enumerate(urls, start=1):
        print(f"\n[{idx}/{len(urls)}] Scraping: {url}")
        try:
            resp = session.get(url, timeout=10)
            resp.raise_for_status()
        except requests.RequestException as e:
            print(f"  ❌ Error fetching {url}: {e}")
            continue

        soup = BeautifulSoup(resp.text, "html.parser")
        title_tag = soup.find("h1")
        title = title_tag.get_text(strip=True) if title_tag else "No Title"
        paragraphs = soup.find_all("p", limit=50)
        content = title + "\n\n" + "\n".join(p.get_text(strip=True) for p in paragraphs).strip()

        print("📄 Snippet:", content[:snippet_len].replace("\n", " "), "...\n")

        metadata = {
            "source": url,
            "title": title,
            "domain": urlparse(url).netloc,
            "scraped_at": datetime.utcnow().isoformat()
        }

        doc = Document(page_content=content, metadata=metadata)
        documents.append(doc)
        time.sleep(delay)

    return documents

# 🏁 Scrape all pages
docs = scrape_content(urls, delay=2)
print(f"\n🧩 Preparing {len(docs)} documents for vector storage...")

# ✂️ Step 3: Chunk content
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
chunks = splitter.split_documents(docs)

# 🧠 Step 4: Embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# 🗃️ Step 5: Store in ChromaDB
persist_directory = "./chroma_techify"
vectordb = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=persist_directory)
vectordb.persist()

print(f"✅ Stored {len(chunks)} chunks in ChromaDB   at: {persist_directory}")

# 🔍 Step 6: View first 5 stored embeddings and metadata
print("\n🔢 Fetching first 5 stored vector embeddings...\n")
stored_data = vectordb._collection.get(include=["embeddings", "documents", "metadatas"])

for i in range(min(5, len(stored_data['ids']))):
    print(f"📄 Document {i + 1}")
    print(f"🧠 Embedding (first 10 dims): {stored_data['embeddings'][i][:10]}")
    content_snippet = stored_data['documents'][i][:150].replace('\n', ' ')
    print(f"📑 Content Snippet: {content_snippet}")
    print(f"🔗 Metadata: {stored_data['metadatas'][i]}")
    print("-" * 80)


🔗 Using 6 URLs:
  • https://techifysolutions.com/industries/
  • https://techifysolutions.com/industries/information-technology/
  • https://techifysolutions.com/industries/real-estate/
  • https://techifysolutions.com/industries/retail/
  • https://techifysolutions.com/industries/automobiles/
  • https://techifysolutions.com/industries/healthcare-and-wellness/

[1/6] Scraping: https://techifysolutions.com/industries/
  ⚠️  No meaningful content found for https://techifysolutions.com/industries/. Skipping.

[2/6] Scraping: https://techifysolutions.com/industries/information-technology/
📄 Snippet: No Title  DevOps Automation Platform ...


[3/6] Scraping: https://techifysolutions.com/industries/real-estate/
📄 Snippet: No Title  DevOps Automation Platform ...


[4/6] Scraping: https://techifysolutions.com/industries/retail/
📄 Snippet: No Title  DevOps Automation Platform ...


[5/6] Scraping: https://techifysolutions.com/industries/automobiles/
📄 Snippet: No Title  DevOps Automation Plat

TypeError: LanceDb.__init__() got an unexpected keyword argument '__path__'

In [ ]:
# import os
# import bs4
# import requests
# from langchain_community.document_loaders import WebBaseLoader
# from dotenv import load_dotenv

# bs4_strainer = bs4.SoupStrainer(class_=('content-area'))
# loader = WebBaseLoader(
#     web_paths = ('https://techifysolutions.com/'),
#     bs_kwargs={'parse only': bs4_strainer}
# )
# docs = loader.load()

In [13]:
import os
from bs4 import BeautifulSoup
import requests

requests.get('https://techifysolutions.com/').text

'<!doctype html><html lang="en-US" prefix="og: https://ogp.me/ns#"><head><link rel="stylesheet" id="ao_optimized_gfonts" href="https://fonts.googleapis.com/css?family=Barlow:italic00,200,300,400,500,600,700,800,900%7CRoboto:italic00,300,400,500,700,900&amp;display=swap"><link media="all" href="https://techifysolutions.com/wp-content/cache/autoptimize/css/autoptimize_bab06cfec9f816d487a8e7e7f4478a26.css" rel="stylesheet"><title>Your IT Services &amp; Solutions Partner | AI-Powered Solutions</title> <!-- Meta Pixel Code --> <script defer src="data:text/javascript;base64,CiFmdW5jdGlvbihmLGIsZSx2LG4sdCxzKQp7aWYoZi5mYnEpcmV0dXJuO249Zi5mYnE9ZnVuY3Rpb24oKXtuLmNhbGxNZXRob2Q/Cm4uY2FsbE1ldGhvZC5hcHBseShuLGFyZ3VtZW50cyk6bi5xdWV1ZS5wdXNoKGFyZ3VtZW50cyl9OwppZighZi5fZmJxKWYuX2ZicT1uO24ucHVzaD1uO24ubG9hZGVkPSEwO24udmVyc2lvbj0nMi4wJzsKbi5xdWV1ZT1bXTt0PWIuY3JlYXRlRWxlbWVudChlKTt0LmFzeW5jPSEwOwp0LnNyYz12O3M9Yi5nZXRFbGVtZW50c0J5VGFnTmFtZShlKVswXTsKcy5wYXJlbnROb2RlLmluc2VydEJlZm9yZSh0LHMpfSh3aW5kb3csIGRvY

In [24]:
pip install chromadb==0.3.29

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/2.3 MB ? eta -:--:--
   ---------------------------------------- 2.3/2.3 MB 21.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.4 MB ? eta -:--:--
   ----------- ---------------------------- 3.1/11.4 MB 15.3 MB/s eta 0:00:01
   ----------------------- ---------------- 6.6/11.4 MB 15.5 MB/s eta 0:00:01
   ---------------------------------- ----- 9.7/11.4 MB 15.1 MB/s eta 0:00:01
   ---------------------------------------- 11.4/11.4 MB 14.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.7 MB ? eta -:--:--
   ---------------------------------------  3.7/3.7 MB 16.8 MB/s eta 0:0

  error: subprocess-exited-with-error
  
  × Building wheel for hnswlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [5 lines of output]
      running bdist_wheel
      running build
      running build_ext
      building 'hnswlib' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for hnswlib
ERROR: Failed to build installable wheels for some pyproject.toml based projects (hnswlib)


In [26]:
pip install --upgrade agno


   ---------------------------------------- 0.0/948.1 kB ? eta -:--:--
   ---------------------------------------- 0.0/948.1 kB ? eta -:--:--
   ---------------------- ----------------- 524.3/948.1 kB 1.9 MB/s eta 0:00:01
   --------------------------------- ------ 786.4/948.1 kB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 948.1/948.1 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: agno
    Found existing installation: agno 1.7.1
    Uninstalling agno-1.7.1:
      Successfully uninstalled agno-1.7.1
Note: you may need to restart the kernel to use updated packages.


In [32]:
from agno.vectordb.chroma import ChromaDb

class CustomChromaDb(ChromaDb):
    def validate_filters(self, filters):
        """
        Stub for filter validation.
        AGNO expects this method to exist for compatibility.
        You can add custom validation logic here if needed.
        """
        # For now, always return True (accept all filters).
        return True


In [ ]:
import os
from dotenv import load_dotenv
from agno.agent import Agent
from agno.models.groq import Groq
from agno.vectordb import chroma  
# 🌱 Load environment variables
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY_temp_2")
if not GROQ_API_KEY:
    raise EnvironmentError("❌ Missing GROQ_API_KEY_temp_2 in .env")

# 🧠 Connect to FAISS vector database (stored locally in ./faiss_techify)
knowledge_base = CustomChromaDb(
    collection="techify",
    path="Tasks_and_codes/Day_16_20/chroma_techify",
    persistent_client=True,
    embedder="sentence-transformers/all-MiniLM-L6-v2",
    add_references=False
)

# 🤖 Initialize the AGNO RAG agent with GROQ model
agent = Agent(
    model=Groq(id="deepseek-r1-distill-llama-70b", api_key=GROQ_API_KEY),
    knowledge=knowledge_base,
    search_knowledge=True,
    add_references=False,  # no reference metadata generation
    markdown=True,
    instructions="Answer very briefly."
)

# 💬 Interactive loop
if __name__ == "__main__":
    print("\n🚀 Welcome to the Techify Q&A Agent powered by AGNO + GROQ + FAISS\nType 'exit' to quit.\n")
    while True:
        query = input("You: ").strip()
        if query.lower() in ("exit", "quit"):
            print("👋 Exiting Techify Agent. Goodbye!")
            break

        try:
            response = agent.run(query)
            print("\nAssistant:\n", response.content, "\n")
        except Exception as e:
            print("\nAssistant:\n It seems there's an issue accessing the information. Please try again later.\n")



🚀 Welcome to the Techify Q&A Agent powered by AGNO + GROQ + FAISS
Type 'exit' to quit.



WARNING  Could not run function search_knowledge_base(query=...)

ERROR    cannot unpack non-iterable bool object                                                                    
         Traceback (most recent call last):                                                                        
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\agno\tools\function.py", line 600, in    
         execute                                                                                                   
             def execute(self) -> FunctionExecutionResult:                                                         
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^                                                        
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\pydantic\_internal\_validate_call.py",   
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\pydantic\_internal\_validate_call.py",   
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\pydantic\_internal\_validate_call.py",   
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\pydantic\_internal\_validate_call.py",   
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\agno\agent\agent.py", line 6369, in      
         search_knowledge_base                                                                                     
             log_warning("Unable to parse response with parser model")                                             
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                                            
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\agno\agent\agent.py", line 5143, in      
         get_relevant_docs_from_knowledge                                                                          
             except Exception:                                                                                     
         TypeError: cannot unpack non-iterable bool object

WARNING  Could not run function search_knowledge_base(query=...)

ERROR    cannot unpack non-iterable bool object                                                                    
         Traceback (most recent call last):                                                                        
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\agno\tools\function.py", line 600, in    
         execute                                                                                                   
             def execute(self) -> FunctionExecutionResult:                                                         
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^                                                        
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\pydantic\_internal\_validate_call.py",   
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\pydantic\_internal\_validate_call.py",   
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\pydantic\_internal\_validate_call.py",   
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\pydantic\_internal\_validate_call.py",   
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\agno\agent\agent.py", line 6369, in      
         search_knowledge_base                                                                                     
             log_warning("Unable to parse response with parser model")                                             
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                                            
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\agno\agent\agent.py", line 5143, in      
         get_relevant_docs_from_knowledge                                                                          
             except Exception:                                                                                     
         TypeError: cannot unpack non-iterable bool object

...

WARNING  Could not run function search_knowledge_base(query=...)

ERROR    cannot unpack non-iterable bool object                                                                    
         Traceback (most recent call last):                                                                        
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\agno\tools\function.py", line 600, in    
         execute                                                                                                   
             def execute(self) -> FunctionExecutionResult:                                                         
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^                                                        
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\pydantic\_internal\_validate_call.py",   
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\pydantic\_internal\_validate_call.py",   
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\pydantic\_internal\_validate_call.py",   
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\pydantic\_internal\_validate_call.py",   
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\agno\agent\agent.py", line 6369, in      
         search_knowledge_base                                                                                     
             log_warning("Unable to parse response with parser model")                                             
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                                            
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\agno\agent\agent.py", line 5143, in      
         get_relevant_docs_from_knowledge                                                                          
             except Exception:                                                                                     
         TypeError: cannot unpack non-iterable bool object

WARNING  Could not run function search_knowledge_base(query=...)

ERROR    cannot unpack non-iterable bool object                                                                    
         Traceback (most recent call last):                                                                        
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\agno\tools\function.py", line 600, in    
         execute                                                                                                   
             def execute(self) -> FunctionExecutionResult:                                                         
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^                                                        
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\pydantic\_internal\_validate_call.py",   
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\pydantic\_internal\_validate_call.py",   
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\pydantic\_internal\_validate_call.py",   
         line 39, in wrapper_function                                                                              
             return wrapper(*args, **kwargs)                                                                       
                    ^^^^^^^^^^^^^^^^^^^^^^^^                                                                       
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\pydantic\_internal\_validate_call.py",   
         line 136, in __call__                                                                                     
             res = self.__pydantic_validator__.validate_python(pydantic_core.ArgsKwargs(args, kwargs))             
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^             
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\agno\agent\agent.py", line 6369, in      
         search_knowledge_base                                                                                     
             log_warning("Unable to parse response with parser model")                                             
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^                                            
           File "c:\Users\KUSHAL\anaconda3\envs\fapienv\Lib\site-packages\agno\agent\agent.py", line 5143, in      
         get_relevant_docs_from_knowledge                                                                          
             except Exception:                                                                                     
         TypeError: cannot unpack non-iterable bool object


Assistant:
 Operation cancelled by user 



In [22]:
from agno.vectordb.faiss import FaissDb


ModuleNotFoundError: No module named 'agno.vectordb.faiss'